### Face Identification Example

The notebook shows how to use face recognition with Aana SDK. Face recognition uses three separate deployments:
1. **Face Detection**, which returns bounding boxes and face landmarks (keypoints) for each detected face
2. **Face Feature Extraction**, which for a given image and face landmarks returns a face feature that can be used to compare face similarities.
3. **Face Database**, which uses 1 and 2 to extract reference faces and populate a reference face database that can be used to search for known identities across image/video collections.

In [1]:
# For onnx GPU support (face detection model), execute this code
!pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

Looking in indexes: https://pypi.org/simple, https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


#### Load the face detection deployment

In [2]:
import os

from aana.deployments.aana_deployment_handle import AanaDeploymentHandle
from aana.deployments.face_detection_deployment import (
    FaceDetectorConfig,
    FaceDetectorDeployment,
)

from aana.sdk import AanaSDK

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

aana_app = AanaSDK()
aana_app.connect(show_logs=False)

# Instantiate and register the face detection deployment
face_detector_deployment = FaceDetectorDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.1},
    user_config=FaceDetectorConfig(
        nms_thresh=0.4,
        batch_size=4,
        input_size=640,
    ).model_dump(mode="json"),
)

aana_app.register_deployment("face_detector", face_detector_deployment, deploy=True)
facedetector_handle = await AanaDeploymentHandle.create("face_detector")

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-17 06:51:03,500	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
2024-07-17 06:51:07,593	WARNING services.py:2009 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free u

#### Load the face feature extraction deployment

In [3]:
from aana.deployments.face_featureextraction_deployment import (
    FacefeatureExtractorConfig,
    FacefeatureExtractorDeployment,
)

FACEFEATURE_MODEL = "ir_101_webface4M"  # Name of the face feature model to be used. This has to be the same one for face feature extraction deployment and reference face database.

# Instantiate and register the face feature extraction deployment
facefeat_extractor_deployment = FacefeatureExtractorDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.2},
    user_config=FacefeatureExtractorConfig(
        feature_extractor_name=FACEFEATURE_MODEL,
        min_face_norm=19.0,
    ).model_dump(mode="json"),
)

aana_app.register_deployment(
    "facefeat_extractor", facefeat_extractor_deployment, deploy=True
)
facefeat_handle = await AanaDeploymentHandle.create("facefeat_extractor")

The new client HTTP config differs from the existing one in the following fields: ['location']. The new HTTP config is ignored.
2024-07-17 06:51:21,542	INFO handle.py:126 -- Created DeploymentHandle 'mm6lgbwf' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').
2024-07-17 06:51:21,544	INFO handle.py:126 -- Created DeploymentHandle '4cqnmqrh' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').
2024-07-17 06:51:28,616	INFO handle.py:126 -- Created DeploymentHandle 'oq4hqr9v' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').
2024-07-17 06:51:28,618	INFO api.py:584 -- Deployed app 'facefeat_extractor' successfully.
2024-07-17 06:51:28,630	INFO handle.py:126 -- Created DeploymentHandle 'ogn9hdzs' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').


2024-07-17 06:51:28,645	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor'): {'9m26ia5j'}.
2024-07-17 06:51:38,277	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='FaceDatabaseDeployment', app='face_database'): {'ftlj9gfc'}.


#### Load the reference face database deployment

In [4]:
from aana.deployments.face_database_deployment import (
    FaceDatabaseConfig,
    FaceDatabaseDeployment,
)
from aana.configs.settings import settings

face_database_deployment = FaceDatabaseDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.1},
    user_config=FaceDatabaseConfig(
        face_threshold=1.18,
        face_features_directory=settings.artifacts_dir / "face_features_database",
        feature_extractor_name=FACEFEATURE_MODEL,
    ).model_dump(mode="json"),
)

aana_app.register_deployment("face_database", face_database_deployment, deploy=True)

facedatabase_handle = await AanaDeploymentHandle.create("face_database")

The new client HTTP config differs from the existing one in the following fields: ['location']. The new HTTP config is ignored.
2024-07-17 06:51:31,182	INFO handle.py:126 -- Created DeploymentHandle 'tnzpda8t' for Deployment(name='FaceDatabaseDeployment', app='face_database').
2024-07-17 06:51:31,184	INFO handle.py:126 -- Created DeploymentHandle 'p3fwj0u2' for Deployment(name='FaceDatabaseDeployment', app='face_database').


2024-07-17 06:51:38,252	INFO handle.py:126 -- Created DeploymentHandle '3ea7ghc7' for Deployment(name='FaceDatabaseDeployment', app='face_database').
2024-07-17 06:51:38,254	INFO api.py:584 -- Deployed app 'face_database' successfully.
2024-07-17 06:51:38,265	INFO handle.py:126 -- Created DeploymentHandle 'p259lxjg' for Deployment(name='FaceDatabaseDeployment', app='face_database').


### Test Deployments

#### Run face detection

In [5]:
from aana.core.models.image import Image
from pathlib import Path

image = Image(
    path=Path(
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000101.jpg"
    )
)

detector_output = await facedetector_handle.predict([image])

print(detector_output)

2024-07-17 06:51:41,334	INFO handle.py:126 -- Created DeploymentHandle '29qqxp9n' for Deployment(name='FaceDetectorDeployment', app='face_detector').


{'bounding_boxes': [array([[415.59457   ,  96.23158   , 481.6673    , 184.54308   ,
          0.87722456],
       [188.31061   ,  60.958458  , 272.11404   , 174.89207   ,
          0.7909324 ]], dtype=float32)], 'keypoints': [array([[[443.3935 , 131.3212 ],
        [471.51398, 133.13217],
        [462.67175, 147.79501],
        [447.37967, 164.56483],
        [467.67795, 165.76624]],

       [[232.06563, 103.42656],
        [264.94907, 106.04597],
        [261.4791 , 123.83058],
        [233.09895, 144.04256],
        [261.0374 , 146.01912]]], dtype=float32)]}


#### Run Face feature extraction using the output of the face detector

In [6]:
keypoints = detector_output["keypoints"]
facefeat_output = await facefeat_handle.predict([image], keypoints)

print(facefeat_output)

2024-07-17 06:51:46,511	INFO handle.py:126 -- Created DeploymentHandle 'b05r8988' for Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').


{'facefeats_per_image': [{'face_feats': array([[-0.04401124, -0.0272526 ,  0.01495183, ..., -0.06465618,
         0.0389987 ,  0.0128512 ],
       [ 0.01096947, -0.02446904,  0.03266014, ..., -0.01851118,
         0.01181385,  0.02578571]], dtype=float32), 'norms': array([[21.607306],
       [23.78863 ]], dtype=float32)}]}


#### Search the reference face database with the face features we extracted above

In [7]:
all_identities_in_database = await facedatabase_handle.get_all_identities()

print(
    "There are {} identities in the reference face database.".format(
        len(all_identities_in_database)
    )
)
print("The first 10 are: {}".format(all_identities_in_database[0:10]))

2024-07-17 06:51:50,171	INFO handle.py:126 -- Created DeploymentHandle '5glsy2hd' for Deployment(name='FaceDatabaseDeployment', app='face_database').


There are 32898 identities in the reference face database.
The first 10 are: ['Paola Minaccioni', 'Yann Hnautra', 'Anna Raadsveld', 'Charlotte Arnold', 'Dustin Clare', 'Erica Carroll', 'J.A. Bayona', 'Kathryn Bigelow', 'Yon González', 'Gil Darnell']


In [9]:
recognized_faces = await facedatabase_handle.search(
    facefeat_output["facefeats_per_image"][0]["face_feats"]
)
print(recognized_faces)

2024-07-17 06:52:08,475	INFO handle.py:126 -- Created DeploymentHandle 'wmb2bj6e' for Deployment(name='FaceDatabaseDeployment', app='face_database').


{'identities': [{'person_id': 'Donna Murphy', 'image_id': '587eed19-0fd7-4f1b-8911-e347f86b1bc8', 'distance': 0.7868431806564331}, {'person_id': 'Patrick Stewart', 'image_id': '00a35c5b-aacf-4574-8219-9c4cb36486a4', 'distance': 0.7150369882583618}]}


## DEBUG (Ignore anything below here)

In [3]:
from aana.configs.settings import settings
import importlib
import aana.deployments.face_database_deployment

importlib.reload(aana.deployments.face_database_deployment)
from aana.deployments.face_database_deployment import (
    FaceDatabaseConfig,
    FaceDatabaseDeployment,
)


facedatabase_config = FaceDatabaseConfig(
    face_threshold=1.18,
    face_features_directory=settings.artifacts_dir / "face_features_database",
    feature_extractor_name=FACEFEATURE_MODEL,
)
facedatabase_deployment = FaceDatabaseDeployment()
await facedatabase_deployment.apply_config(facedatabase_config.model_dump(mode="json"))

In [7]:
import numpy as np

facefeats_ir101_web4M = np.load(
    "/nas/dominic/AanaFaceEval/reference_facedict_aanasdk_webface4M_r101.npy",
    allow_pickle=True,
).item()

In [8]:
import tqdm

for k in tqdm.tqdm(range(len(facefeats_ir101_web4M["face_features"]))):
    ffeat = facefeats_ir101_web4M["face_features"][k]
    name_ = facefeats_ir101_web4M["person_names"][k]
    img_id = (
        facefeats_ir101_web4M["paths_to_image"][k].split("|||")[0].split("/")[-1]
    )  # eg '000744f4-4131-44db-a569-ca211fa55a48'

    res = await facedatabase_handle.add_reference_face(
        ffeat, person_name=name_, image_id=img_id
    )

  0%|          | 0/32898 [00:00<?, ?it/s]


NameError: name 'facedatabase_deployment' is not defined

In [45]:
# Sample image paths
image_paths = [
    "image1.jpg",
    "image2.jpg",
    "image3.jpg",
    "image4.jpg",
    "image5.jpg",
    "image6.jpg",
    "image7.jpg",
    "image8.jpg",
    "image9.jpg",
    "image10.jpg",
]

# Populate the data dictionary
data = {"images": [{"path": im_path} for im_path in image_paths[0:10]]}

# Print the result
print(data)

{'images': [{'path': 'image1.jpg'}, {'path': 'image2.jpg'}, {'path': 'image3.jpg'}, {'path': 'image4.jpg'}, {'path': 'image5.jpg'}, {'path': 'image6.jpg'}, {'path': 'image7.jpg'}, {'path': 'image8.jpg'}, {'path': 'image9.jpg'}, {'path': 'image10.jpg'}]}


In [49]:
import requests, json


def extract_face_feats(im_path):
    data = {
        "images": [{"path": im_path}, {"path": im_path}],
    }

    response = requests.post(
        "http://127.0.0.1:8000/recognize_faces",
        data={"body": json.dumps(data)},
        stream=False,
    )
    res = response.json()

    return res
    # return res['face_features_per_image'][0]['face_feats'], res['face_features_per_image'][0]['norms']


def extract_face_feats_batched(im_paths):
    data = {"images": [{"path": im_path} for im_path in im_paths]}

    print(data)

    response = requests.post(
        "http://127.0.0.1:8000/recognize_faces",
        data={"body": json.dumps(data)},
        stream=False,
    )
    res = response.json()

    return res

    # face_feats = [res['face_features_per_image'][k]['face_feats'] for k in range(len(res['face_features_per_image']))]
    # norms = [res['face_features_per_image'][k]['norms'] for k in range(len(res['face_features_per_image']))]

    # return face_feats, norms

In [40]:
res = extract_face_feats(
    "/nas/dominic/AanaFaceEval/identities_reference/default/000bb305-88c1-4490-977f-59e66b755cd4|||Ilse Neubauer.jpg"
)

In [55]:
data = {"images": [{"path": im_path} for im_path in image_paths[0:10]]}

print(data)

{'images': [{'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000744f4-4131-44db-a569-ca211fa55a48|||Keita Machida.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0007970d-a5d9-4791-bb47-9c52d4cf4473|||Caroleen Feeney.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0009aaba-e0da-46c3-a557-72d318ce0bd8|||Ewa Fröling.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000b66a3-5009-4513-aa13-535202c679b3|||Marielle Heller.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000bb305-88c1-4490-977f-59e66b755cd4|||Ilse Neubauer.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000d385b-ac12-41b0-9692-7abff6c5546a|||Michel Joelsas.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00112729-6837-424c-bdf5-1709b9b38cba|||Patricia Hodge.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00115272-41a7-4b22-bb5a-ea079aa2dea6|||Gilli

In [72]:
res = extract_face_feats_batched(image_paths[13:19])

{'images': [{'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001c3bcf-3ab0-48cc-b805-c8da6ed3ba6a|||Eryk Lubos.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001d0484-4b83-42da-bff9-145bcc46359b|||Sanjjanaa Archana Galrani.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001e04f1-a8f4-45e0-a85d-59d8bf06f12a|||Gerald Okamura.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0021297f-e5f7-4965-a0cd-bb42c4d70732|||Laurie Metcalf.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00227c4a-8cf5-491f-823d-4bd6251efac1|||Luís Melo.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0022e2e0-f72b-4470-af26-0b93527b3ebf|||Marie-Mae van Zuilen.jpg'}]}


In [73]:
res

{'error': 'IndexError',
 'message': 'list index out of range',
 'data': {},
 'stacktrace': '\x1bray::ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment.handle_request_with_rejection()\x1b (pid=9498, ip=172.17.0.3, actor_id=2d13ca81aac0a71ca1ac728801000000, repr=<ray.serve._private.replica.ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment object at 0x7f035bcd8af0>)\n    yield await self._user_callable_wrapper.call_user_method(\n  File "/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/ray/serve/_private/replica.py", line 1150, in call_user_method\n    raise e from None\nray.exceptions.RayTaskError: \x1bray::ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment.handle_request_with_rejection()\x1b (pid=9498, ip=172.17.0.3)\n  File "/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/ray/serve/_private/utils.py", line 168, in wrap_to_ray_error\n    raise exce

In [2]:
import tqdm
import glob
import os


reference_faces_dict_30K_webface4M_r101 = {
    "person_names": [],
    "paths_to_image": [],
    "face_features": [],
    "norms": [],
}

failed_faces = []

image_paths = sorted(
    glob.glob("/nas/dominic/AanaFaceEval/identities_reference/default/*.jpg")
)

for image_filepath in tqdm.tqdm(image_paths):
    face_feats, norms = extract_face_feats(image_filepath)

    # import pdb; pdb.set_trace()

    if face_feats is not None and len(face_feats) == 1 and norms[0][0] >= 19.0:
        reference_faces_dict_30K_webface4M_r101["person_names"].append(
            os.path.basename(image_filepath)[0:-4].split("|||")[-1]
        )
        reference_faces_dict_30K_webface4M_r101["paths_to_image"].append(image_filepath)
        reference_faces_dict_30K_webface4M_r101["face_features"].append(face_feats[0])
        reference_faces_dict_30K_webface4M_r101["norms"].append(norms[0][0])
    else:
        failed_faces.append([image_filepath, norms[0][0]])

reverse_reference_faces_dict_30K_webface4M_r101 = {
    reference_faces_dict_30K_webface4M_r101["person_names"][
        k
    ]: reference_faces_dict_30K_webface4M_r101["paths_to_image"][k]
    for k in range(len(reference_faces_dict_30K_webface4M_r101["person_names"]))
}

  0%|          | 28/33541 [00:07<2:38:11,  3.53it/s]


KeyboardInterrupt: 